In [24]:
import numpy as np
import matplotlib.pyplot as plt
import math

import particles
from particles import distributions as dists
from particles import state_space_models as ssm

np.random.seed(1234)

In [ ]:
d = 2
K = 1000


prior_pi = dists.Normal(loc=0.0, scale=0.1)

# Draw K particles (y0, x0) from the prior pi
x0 = np.zeros((K, d))
y0 = np.zeros((K, d))
for k in range(K):
    x0[k] = prior_pi.rvs(size=d)
    y0[k] = prior_pi.rvs(size=d)


def phi(x, cst):
    # cst: (d, )
    # x: (K, d)
    return cst * np.exp(x)  # (d, ) * (K, d) = (K, d)


phit = phi(x0, phi0)

times = np.arange(0, 10.1, 0.1)

(2,)
[ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1  1.2  1.3
  1.4  1.5  1.6  1.7  1.8  1.9  2.   2.1  2.2  2.3  2.4  2.5  2.6  2.7
  2.8  2.9  3.   3.1  3.2  3.3  3.4  3.5  3.6  3.7  3.8  3.9  4.   4.1
  4.2  4.3  4.4  4.5  4.6  4.7  4.8  4.9  5.   5.1  5.2  5.3  5.4  5.5
  5.6  5.7  5.8  5.9  6.   6.1  6.2  6.3  6.4  6.5  6.6  6.7  6.8  6.9
  7.   7.1  7.2  7.3  7.4  7.5  7.6  7.7  7.8  7.9  8.   8.1  8.2  8.3
  8.4  8.5  8.6  8.7  8.8  8.9  9.   9.1  9.2  9.3  9.4  9.5  9.6  9.7
  9.8  9.9 10. ]


In [48]:
A = np.diag([0.8, 0.2])
V = np.array([[0.5, 0.1], [0.1, 0.3]])
PHI = np.random.randn(d)


def Gamma(tau):
    gamma = np.zeros((d, d))
    vv = V @ V.T

    for i in range(d):
        ai = A[i, i]
        for j in range(d):
            aj = A[j, j]
            gamma[i, j] = 1 / (ai + aj) * (1 - math.exp(-(ai + aj) * tau)) * vv[i, j]
    return gamma

In [50]:
# First Step: Draw half bid-ask spreads
def step1(m, tau, x):
    g = Gamma(tau[m + 1] - tau[m])  # (d, d)
    mu = np.exp(-A * (tau[m + 1] - tau[m])) @ x.T  # (d, d) @ (d, K) -> (d, K)

    # For each particle, draw the next state
    hat_x = dists.MvNormal(loc=mu.T, cov=g).rvs(size=K)  # (K, d)
    hat_phi = phi(hat_x, PHI)  # (K, d)

    return hat_x, hat_phi


# test
hatx, hatphi = step1(0, times, x0)
print(hatx.shape, hatphi.shape)

(1000, 2) (1000, 2)


In [ ]:
# Step 2: Compute weights
def step2(m, tau, J, obs, y, hatphi, sigma_i=0.1, sigma_noise=0.1, alpha=0.8):
    w = np.zeros(K)
    i, J = J

    hatphi = hatphi[:, i]  # (K, d) -> (K, )
    y = y[:, i]  # (K, d) -> (K, )
    obs = obs[:, i]  # (K, d) -> (K, )

    if J == 0:
        num = obs + hatphi - y  # (K, ) + (K, d) - (K, d) -> (K, d)
        den = np.sqrt(sigma_i**2 * (tau[m + 1] - tau[m]) + sigma_noise**2)  # scalar

        w = dists.Normal(loc=0.0, scale=1.0).pdf(num / den)  # (K, d) / scalar -> (K, d)

    if J == 1:
        num = obs - hatphi - y
        den = np.sqrt(sigma_i**2 * (tau[m + 1] - tau[m]) + sigma_noise**2)

        w = dists.Normal(loc=0.0, scale=1.0).pdf(num / den)

    if J == 2:
        num = obs + hatphi - y
        den = np.sqrt(sigma_i**2 * (tau[m + 1] - tau[m]) + sigma_noise**2)

        w = dists.Normal(loc=0.0, scale=1.0).cdf(-num / den)

    if J == 3:
        num = obs - hatphi - y
        den = np.sqrt(sigma_i**2 * (tau[m + 1] - tau[m]) + sigma_noise**2)

        w = dists.Normal(loc=0.0, scale=1.0).cdf(num / den)

    if J == 4:
        num1 = obs + alpha * hatphi - y
        num2 = obs - alpha * hatphi - y
        den = np.sqrt(sigma_i**2 * (tau[m + 1] - tau[m]) + sigma_noise**2)

        w = dists.Normal(loc=0.0, scale=1.0).cdf(num1 / den) - dists.Normal(
            loc=0.0, scale=1.0
        ).cdf(num2 / den)

    return w / np.sum(w)

In [ ]:
# Step 3: Resample
def step3(w, hatx, y):
    idx = np.random.choice(K, size=K, replace=True, p=w)
    hatx = hatx[idx]
    y = y[idx]
    hatphi = phi(hatx, PHI)
    return y, hatx, hatphi